In [1]:
from ipywidgets import Label, VBox, HBox
from ipywidgets import FloatSlider, IntSlider, Checkbox
from ipywidgets import interactive_output

import math
import torch
import numpy as np

from torch import nn
from torch.nn import functional as F
from torchvision.utils import make_grid

from PIL import Image
from sdf_renderer import SDFRenderer

CAM_UI_DEFAULTS = {
    'azimuth': {'value': 0, 'min': -180, 'max': 180, 'step': 0.1},
    'inclination': {'value': 0, 'min': -180, 'max': 180, 'step': 0.1},
    'distance': {'value': 3.6, 'min': 0, 'max': 100, 'step': 0.1}
}


def build_cam_ui(callback, extras=None, **kwargs):
    global_kwargs = {'continuous_update': kwargs.get('continuous_update', True)}
    get_named_value = lambda key, name: kwargs.get(f'{key}_{name}', CAM_UI_DEFAULTS[key][name])

    control_by_key = {}
    for k in ('azimuth', 'inclination', 'distance'):
        local_kwargs = dict(global_kwargs)
        local_kwargs.update({name: get_named_value(k, name) for name in CAM_UI_DEFAULTS[k]})
        control_by_key[k] = FloatSlider(description=k.capitalize(), **local_kwargs)

    if extras is not None: control_by_key.update(extras)

    control_list = list(control_by_key.values())
    out = interactive_output(callback, control_by_key)

    return HBox([out, VBox(control_list)]), control_by_key


def tensor_to_image(tensor, *args, **kwargs):
    if tensor.ndimension() == 4:
        tensor = make_grid(tensor, *args, **kwargs)
    tensor = tensor.permute(1, 2, 0).mul(255.)
    return tensor.data.byte().cpu().numpy()


def spherical_to_cartesian(input):
    r, theta, phi = input
    r = max(r, 0.0)

    theta = theta * math.pi / 180
    phi = phi * math.pi / 180

    x = r * math.sin(theta) * math.cos(phi)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)

    return x, y, z


def _float_to_tensor(value, points):
    bs = points.shape[0]
    if isinstance(value, float):
        value = torch.full([bs, 1], value, device=points.device)
    elif torch.is_tensor(value) and value.shape[0] == 1:
        value = value.view(1, 1).expand(bs, 1)
    return value

In [2]:
sdf_renderer = SDFRenderer(
    image_size=300, background_color=[1.0, 1.0, 1.0], 
    shading='distance', hit_threshold=0.05, iterations=20, fov=45
).cuda()

In [ ]:
def texture_model(surface_points):
    surface_points = surface_points.view(-1, 3)
    return torch.ones_like(surface_points), torch.ones_like(surface_points[:, 0])

def torus(p, s0, s1):
    s0 = _float_to_tensor(s0, p)
    s1 = _float_to_tensor(s1, p)

    q = torch.cat([p[:, [0, 2]].norm(2, -1, True) - s0, p[:, [1]]], -1)
    return q.norm(2, -1, True) - s1
    
def f(azimuth, inclination, distance, s0, s1):
    with torch.no_grad():
        distance_model = lambda p: torus(p, s0, s1)
        eye = torch.tensor(spherical_to_cartesian((distance, azimuth, inclination)), device='cuda')[None]
        image = sdf_renderer.render_look_at(distance_model, texture_model, None, eye=eye)
        
    image_numpy = tensor_to_image(image)
    display(Image.fromarray(image_numpy))

ui, controls = build_cam_ui(f, extras={
    's0': FloatSlider(description="s0", min=0, max=5, step=.1, value=1, continuous_update=True),
    's1': FloatSlider(description="s1", min=0, max=5, step=.1, value=.4, continuous_update=True)
}, azimuth_value=-84.1, inclination_value=-48.20)

ui

In [11]:
# test display mesh from dataset
import kaolin as kal


mesh = kal.rep.TriangleMesh.from_obj('../data/0_simplified.obj')
sdf_model = kal.conversions.trianglemesh_to_sdf(mesh)
points = torch.rand(100,3)
surface_points = mesh.vertices
# distances = sdf_model(points)
# print("distances", distances)

def eval_query(query):
    distances = directed_distance(query, surface_points, mean=False)
    occ_points = kal.rep.SDF.check_sign(mesh, query)
    if torch.is_tensor(occ_points):
        occ_points = occ_points.cpu().numpy()[0]
    distances[np.where(occ_points)] *= -1
    return distances


def texture_model(surface_points):
    surface_points = surface_points.view(-1, 3)
    print("surface_points", surface_points.shape)
    print(torch.ones_like(surface_points).shape, torch.ones_like(surface_points[:, 0]).shape)
    return torch.ones_like(surface_points), torch.ones_like(surface_points[:, 0])

def torus(p, s0, s1):

    s0 = _float_to_tensor(s0, p)
    s1 = _float_to_tensor(s1, p)
#     print("p", p.shape)
    q = torch.cat([p[:, [0, 2]].norm(2, -1, True) - s0, p[:, [1]]], -1)
#     print("q", q.shape)
    qq = q.norm(2, -1, True) - s1
#     print("qq", qq.shape)
    return q.norm(2, -1, True) - s1
    
def f(azimuth, inclination, distance, s0, s1):
    with torch.no_grad():
        distance_model = lambda p:torus(p, s0, s1) # sdf_model(points) # torus(p, s0, s1)
#         print(torus(torch.tensor, s0, s1))
        eye = torch.tensor(spherical_to_cartesian((distance, azimuth, inclination)), device='cuda')[None]
        image = sdf_renderer.render_look_at(distance_model, texture_model, None, eye=eye)
        
    image_numpy = tensor_to_image(image)
    display(Image.fromarray(image_numpy))

ui, controls = build_cam_ui(f, extras={
    's0': FloatSlider(description="s0", min=0, max=5, step=.1, value=1, continuous_update=True),
    's1': FloatSlider(description="s1", min=0, max=5, step=.1, value=.4, continuous_update=True)
}, azimuth_value=-84.1, inclination_value=-48.20)

ui
